# POC Model

__Install packages__

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
from nltk.corpus import stopwords
import glob
from datetime import datetime
import statistics 
import pickle
import json
from rake_nltk import Rake
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer
from gensim.test.utils import datapath
import math
from collections import Counter
from gensim.models import Word2Vec
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import string
%load_ext autotime

C:\ProgramData\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\ProgramData\Anaconda3\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict


### Data Sources Integration

In [2]:
data = []
for filename in glob.glob('*.csv'):
    data.append(filename)
data

['COPA_blog.csv',
 'COPA_forum_replies.csv',
 'COPA_forum_threads.csv',
 'COPA_Wiki.csv',
 'only contentscore.csv',
 'only wmd score.csv',
 'PreContentScore.csv',
 'processed_df.csv',
 'ratio_comb1.csv',
 'ratio_comb2.csv',
 'ratio_comb3.csv',
 'rerank_result.csv',
 'SearchApiResults.csv',
 'wmd_comb1.csv',
 'wmd_comb2.csv',
 'wmd_comb3.csv',
 'wmd_count_comb1.csv',
 'wmd_count_comb2.csv',
 'wmd_count_comb3.csv',
 'wmd_ratio_comb1.csv',
 'wmd_ratio_comb2.csv',
 'wmd_ratio_comb3.csv',
 'WordPairs.csv']

time: 6.89 ms


In [3]:
wiki = pd.read_csv(data[3])
wiki = wiki[['LastModifiedUtcDate','UserId', 'Title', 'FormattedBody', 'TotalViews','ContentID']]
wiki['LastModifiedUtcDate'] = wiki['LastModifiedUtcDate'].apply(lambda x: x[0:10])
wiki.columns = ['Date', 'UserID', 'Title', 'FormattedBody', 'TotalViews','ContentId']
wiki['Source'] = 'Wiki'
wiki.head()

,Date,UserID,Title,FormattedBody,TotalViews,ContentId,Source
0,2008-12-21,2256,First page,http://www.cirruspilots.org/wikis/sandbox/add....,294,41A72871-00C2-40A8-81D1-E7E7EC515A88,Wiki
1,2009-04-11,3737,Cockpit DVR - Version 4,\r\nThe techology just keeps improving. Here&n...,460,8E1B83F6-70E7-475B-910F-5243923936C3,Wiki
2,2013-12-09,2101,Sandbox,NaN,0,0B0D2199-995F-46FC-BA41-B22EE878105D,Wiki
3,2009-01-04,2256,Test wiki for fatal accident #1,\r\n\r\n\r\nCirrus Fatal Accident #1Sierra Vis...,2533,B11E5754-0659-493A-B1BC-4452EE6846D4,Wiki
4,2008-12-26,2256,Test infobox for CAPS activations,\r\n\r\n\r\nCirrus CAPS Activation #1Lewisvill...,189,CFD32812-6AEB-4219-93A5-CF94EFCCD717,Wiki


time: 88.7 ms


In [4]:
# Load forum replies
forum_replies = pd.read_csv(data[1])
forum_replies = forum_replies[['ThreadReplyDate', 'UserID', 'FormattedBody', 'ContentId']]
forum_replies.columns = ['Date', 'UserID', 'FormattedBody', 'ContentId']
forum_replies['Date'] = forum_replies['Date'].apply(lambda x: x.split(' ')[0])
forum_replies['Source'] = 'Forum'
forum_replies.head(10)

,Date,UserID,FormattedBody,ContentId,Source
0,2008-06-25,3870,You are cordially invited to a Cirrus BBQ & Fu...,3419B3DE-98D8-40DB-BDFE-A1C267B7DCDA,Forum
1,2005-10-23,4255,"Ed,Thank you for the offer. My envelope is on ...",3CC6AA08-9187-41A3-9A6B-0CD9DCA0B563,Forum
2,2005-10-23,3426,If I come over Tuesday do I get to play Igor t...,34569FEC-864A-432C-9AEC-6914E7A05844,Forum
3,2005-10-24,2296,"Ed, My envelope is on the way also! thanks for...",69C3151C-BD6D-4E5A-88A2-7A2F26548923,Forum
4,2005-11-04,3586,"When I returned home from S FL yesterday, I co...",2F983FAD-25B4-4BC9-996C-7F8CEA98A1EF,Forum
5,2005-11-16,2296,"Ed, I got the devices today! Thank you very mu...",086DAB71-8E58-444C-8C8B-3DDD0FAB7A94,Forum
6,2005-11-16,4723,"Eddie,Thank you, I think this is a great contr...",086228CA-24C0-4E24-A349-4607960DB036,Forum
7,2005-11-20,4228,Sending the SASE out today and Thanks!GaynorP....,45B7CD9F-CDDD-4E92-BCBA-16E39999E482,Forum
8,2005-12-05,3586,Latest batch going out tomorrow to:Bruce Goldm...,99C0BC29-66FE-41B7-8429-18146E422BDA,Forum
9,2005-12-06,3586,Sets going out today to:Barry LermanBret Fishe...,91F35389-EA20-477E-A80C-C056B8F55333,Forum


time: 6.53 s


In [5]:
blog = pd.read_csv(data[0])
blog = blog[['PostDate', 'UserId', 'PostName', 'FormattedBody', 'TotalViews','ContentId']]
blog.columns = ['Date', 'UserID', 'Title', 'FormattedBody', 'TotalViews','ContentId']
blog['Date'] = forum_replies['Date'].apply(lambda x: x.split(' ')[0])
blog['Source'] = 'Blog'
blog.head()

,Date,UserID,Title,FormattedBody,TotalViews,ContentId,Source
0,2008-06-25,2101,aviation note,I am way behind on the small-plane aviation ne...,909,C7F8D21E-9E7D-4AE2-BFBF-C18A4A09C4FF,Blog
1,2005-10-23,4869,bienvenue dans le french corner,"<p class=""MsoNormal"">Le lancement du nouveau s...",2493,DD04A896-7DD4-4DB6-807E-9B2E915C055D,Blog
2,2005-10-23,4869,opportunit 233,<p>Si vous &ecirc;tes tenaill&eacute; par l&#3...,1583,08353297-F100-4CD8-BBE8-6EC165263D9B,Blog
3,2005-10-24,4869,un avion en novembre,<p>&nbsp;</p>\r\n<p>Une question qui m&#39;est...,1600,DF62C904-AD9B-4F61-BC41-33693A489631,Blog
4,2005-11-04,4869,volez du c 244 t 233 pauvre,<p>On ne peut pas adh&eacute;rer &agrave; la C...,2204,9593CE5B-6ED9-4CF7-A062-4B173E6F0EA2,Blog


time: 245 ms


In [6]:
# Load forum replies
forum_threads = pd.read_csv(data[2])
forum_threads = forum_threads[['ThreadDate', 'UserID', 'FormattedBody', 'ContentId']]
forum_threads.columns = ['Date', 'UserID', 'FormattedBody', 'ContentId']
forum_threads['Date'] = forum_threads['Date'].apply(lambda x: x.split(' ')[0])
forum_threads['Source'] = 'Forum'
forum_threads.head(10)

,Date,UserID,FormattedBody,ContentId,Source
0,2003-07-19,2648,Amazing flight sim set up....http://members.ch...,097A58B5-778F-4288-9C54-3CF4B21632BC,Forum
1,2003-07-19,2648,Amazing flight sim set up....http://members.ch...,A137B4D7-3A5E-4B5C-B9D3-5C5E726E936C,Forum
2,2003-10-10,2107,Last Tuesday I flew from Burlington NC back ho...,1176A345-874E-46A1-80FD-DD46B3B40AAE,Forum
3,2003-11-19,3037,Another look.,B0CDB01B-BCD7-45DC-99B4-EF3612DEA04C,Forum
4,2004-06-30,2438,There are still a number of M2 registrants tha...,0E4FAB75-96A2-40E5-85BF-F15310AD8361,Forum
5,2004-09-15,3498,"Strangely, ATCSCC status (www.fly.faa.gov) sho...",37F20AE0-A145-4D06-B4E4-8B7BAD438909,Forum
6,2005-02-10,2107,In reply to:I have a question for the group. P...,D789803B-68A3-47F5-9464-6F9A74586D80,Forum
7,2005-02-10,8522,Great question and a great answer. In areas li...,EE704E2D-CAE4-4036-B2E6-C4E7FFD6688B,Forum
8,2005-02-10,3674,"Howard, that's actually a really good question...",2B70A686-6AD1-49F3-81D1-B8E531573A08,Forum
9,2005-03-17,4675,Just received an e-mail advertisement this mor...,E1A69CB1-0073-4821-B330-3EFA7DF73574,Forum


time: 858 ms


In [7]:
df = pd.concat([blog, wiki], ignore_index=True, sort=False)
df = pd.concat([df, forum_threads], ignore_index=True, sort=False)
df = pd.concat([df, forum_replies], ignore_index=True, sort=False)
df.head()

,Date,UserID,Title,FormattedBody,TotalViews,ContentId,Source
0,2008-06-25,2101,aviation note,I am way behind on the small-plane aviation ne...,909.0,C7F8D21E-9E7D-4AE2-BFBF-C18A4A09C4FF,Blog
1,2005-10-23,4869,bienvenue dans le french corner,"<p class=""MsoNormal"">Le lancement du nouveau s...",2493.0,DD04A896-7DD4-4DB6-807E-9B2E915C055D,Blog
2,2005-10-23,4869,opportunit 233,<p>Si vous &ecirc;tes tenaill&eacute; par l&#3...,1583.0,08353297-F100-4CD8-BBE8-6EC165263D9B,Blog
3,2005-10-24,4869,un avion en novembre,<p>&nbsp;</p>\r\n<p>Une question qui m&#39;est...,1600.0,DF62C904-AD9B-4F61-BC41-33693A489631,Blog
4,2005-11-04,4869,volez du c 244 t 233 pauvre,<p>On ne peut pas adh&eacute;rer &agrave; la C...,2204.0,9593CE5B-6ED9-4CF7-A062-4B173E6F0EA2,Blog


time: 280 ms


In [8]:
#Ranking the source, blog is the most important source so have 3
source_dict = {'Blog': 3, 'Forum': 2, 'Wiki': 1}
df['SourceScore'] = df['Source'].map(source_dict)
df.head()

,Date,UserID,Title,FormattedBody,TotalViews,ContentId,Source,SourceScore
0,2008-06-25,2101,aviation note,I am way behind on the small-plane aviation ne...,909.0,C7F8D21E-9E7D-4AE2-BFBF-C18A4A09C4FF,Blog,3
1,2005-10-23,4869,bienvenue dans le french corner,"<p class=""MsoNormal"">Le lancement du nouveau s...",2493.0,DD04A896-7DD4-4DB6-807E-9B2E915C055D,Blog,3
2,2005-10-23,4869,opportunit 233,<p>Si vous &ecirc;tes tenaill&eacute; par l&#3...,1583.0,08353297-F100-4CD8-BBE8-6EC165263D9B,Blog,3
3,2005-10-24,4869,un avion en novembre,<p>&nbsp;</p>\r\n<p>Une question qui m&#39;est...,1600.0,DF62C904-AD9B-4F61-BC41-33693A489631,Blog,3
4,2005-11-04,4869,volez du c 244 t 233 pauvre,<p>On ne peut pas adh&eacute;rer &agrave; la C...,2204.0,9593CE5B-6ED9-4CF7-A062-4B173E6F0EA2,Blog,3


time: 65.9 ms


In [9]:
df[['FormattedBody']] = df[['FormattedBody']].fillna(value='No text.')

time: 334 ms


## Train the word2vec model

In [10]:
# Define functions to do text preprocessing
def remove_html(text):
    soup = BeautifulSoup(text, 'lxml')
    html_free = soup.get_text()
    return html_free

def remove_punctuation(text):
    no_punct = ''.join([c for c in text if c not in string.punctuation])
    return no_punct

stop_words = stopwords.words('english')

def remove_stopwords(text):
    words = [w for w in text if w not in stop_words]
    return words

lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

def handle_character(text):
    text = text.replace('\r\n',' ')
    text = text.replace('\n',' ')
    text = text.replace('\xa0',' ')
    return text

time: 4.99 ms


In [11]:
'''
train model
'''

'\ntrain model\n'

time: 990 µs


# Generate Attributes

__Generate Recency Score__

In [12]:
def  Calculate_RecencyScore(date):
    '''
    Recency Rate = log( 1 + 1/(days between the post date and current date + 1))
    '''
    date_datetime = datetime.strptime(date, '%Y-%m-%d').date()
    rececncy_rate = math.log10(1/(((datetime.date(datetime.now()))-date_datetime).days+1)+1)
    return rececncy_rate
    

df['RecencyRate'] = df['Date'].map(lambda y: Calculate_RecencyScore(y))

time: 9.68 s


__Generate Author Score__

In [13]:
author_list = list(df['UserID'].unique())
author_score_dict = {}
for author in author_list:
    author_score = df[df['UserID'] == author]['SourceScore'].sum()
    author_score_dict[author] = author_score
    
df['AuthorScore'] = df['UserID'].map(author_score_dict)

time: 15.9 s


In [14]:
df.head()

,Date,UserID,Title,FormattedBody,TotalViews,ContentId,Source,SourceScore,RecencyRate,AuthorScore
0,2008-06-25,2101,aviation note,I am way behind on the small-plane aviation ne...,909.0,C7F8D21E-9E7D-4AE2-BFBF-C18A4A09C4FF,Blog,3,0.000100,2051
1,2005-10-23,4869,bienvenue dans le french corner,"<p class=""MsoNormal"">Le lancement du nouveau s...",2493.0,DD04A896-7DD4-4DB6-807E-9B2E915C055D,Blog,3,0.000082,2198
2,2005-10-23,4869,opportunit 233,<p>Si vous &ecirc;tes tenaill&eacute; par l&#3...,1583.0,08353297-F100-4CD8-BBE8-6EC165263D9B,Blog,3,0.000082,2198
3,2005-10-24,4869,un avion en novembre,<p>&nbsp;</p>\r\n<p>Une question qui m&#39;est...,1600.0,DF62C904-AD9B-4F61-BC41-33693A489631,Blog,3,0.000082,2198
4,2005-11-04,4869,volez du c 244 t 233 pauvre,<p>On ne peut pas adh&eacute;rer &agrave; la C...,2204.0,9593CE5B-6ED9-4CF7-A062-4B173E6F0EA2,Blog,3,0.000082,2198


time: 7.97 ms


__Generate Content Score__

In [15]:
queries_df = pd.read_csv("SearchApiResults.csv")
queries_df = queries_df[['Query', 'url', 'ContentId', 'fancy_title']]
queries_df = queries_df[queries_df['ContentId'].isna() != True]
all_query_df = pd.merge(queries_df, df, on = 'ContentId')
all_query_df.head()

,Query,url,ContentId,fancy_title,Date,UserID,Title,FormattedBody,TotalViews,Source,SourceScore,RecencyRate,AuthorScore
0,brakes during taxi,https://forum.cirruspilots.org/t/41040/5,6D601E0C-14D7-41DB-BAEF-3A6333537AB8,Cirrus SR22 Brake problems,2012-11-21,3695,NaN,"[quote user=""David Martin""]\r\nRobert that's w...",NaN,Forum,2,0.000160,19536
1,brakes during taxi,https://forum.cirruspilots.org/t/65942/2,ADCD2721-0351-4DD7-8B07-AE423D3420F0,cirrus sr22 brake wear,2012-12-11,10652,NaN,[8)],NaN,Forum,2,0.000161,19902
2,brakes during taxi,https://forum.cirruspilots.org/t/25175/7,2A423C4C-B3E0-4C9C-95D1-A0A86ACE1465,"Overheated, Failed Brakes",2005-04-20,3156,NaN,"Yes, the other guy had plenty of time to go ar...",NaN,Forum,2,0.000079,2300
3,brakes during taxi,https://forum.cirruspilots.org/t/68466/1,AB750CAD-7103-48DC-BB4A-8B82B792B33E,Brake fire,2013-06-14,14208,NaN,"Got a fresh annual. On taxi, brakes clamped n...",NaN,Forum,2,0.000173,14
4,brakes during taxi,https://forum.cirruspilots.org/t/26578/1,97F47B9B-F055-4F1D-9C56-8B5FF2A3D32C,Brake Failure and Fire,2005-08-12,4281,NaN,"Today, we had the right landing gear in our ai...",NaN,Forum,2,0.000081,340


time: 501 ms


In [16]:
# Process the extracted dataframe's FormattedBody part
all_query_df['FormattedBody'] = all_query_df['FormattedBody'].astype(str)

all_query_df['FormattedBody'] = all_query_df['FormattedBody'].apply(lambda x: remove_html(x))

all_query_df['FormattedBody'] = all_query_df['FormattedBody'].apply(lambda x: remove_punctuation(x))

all_query_df['FormattedBody'] = all_query_df['FormattedBody'].apply(lambda x: handle_character(x))

def str_to_lst(text):
    lst = text.lower().split()
    return lst

all_query_df['FormattedBody'] = all_query_df['FormattedBody'].apply(lambda x: str_to_lst(x))

all_query_df['FormattedBody'] = all_query_df['FormattedBody'].apply(lambda x: remove_stopwords(x))

all_query_df['FormattedBody'] = all_query_df['FormattedBody'].apply(lambda x: word_lemmatizer(x))

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://www.ktts.com/news/169896596.html
 
http://www.news-leader.com/article/DO/20120915/NEWS01/309150044/Small-plane-crash-north-of-Willard-results-in-multiple-fatalities" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://www.azfamily.com/news/local/Plane-crashes-into-building-at-Deer-Valley-Airport-99934759.html?hpt=T2
 " looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


time: 1.94 s


###  Computing the Word Mover's Distance between query and posts

In [18]:
# Load the pre-trained word2vec model with Google News
import gensim.models.keyedvectors as word2vec
model = word2vec.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

time: 1min 50s


In [19]:
all_query_df

,Query,url,ContentId,fancy_title,Date,UserID,Title,FormattedBody,TotalViews,Source,SourceScore,RecencyRate,AuthorScore
0,brakes during taxi,https://forum.cirruspilots.org/t/41040/5,6D601E0C-14D7-41DB-BAEF-3A6333537AB8,Cirrus SR22 Brake problems,2012-11-21,3695,NaN,"[quote, userdavid, martin, robert, thats, mech...",NaN,Forum,2,0.000160,19536
1,brakes during taxi,https://forum.cirruspilots.org/t/65942/2,ADCD2721-0351-4DD7-8B07-AE423D3420F0,cirrus sr22 brake wear,2012-12-11,10652,NaN,[8],NaN,Forum,2,0.000161,19902
2,brakes during taxi,https://forum.cirruspilots.org/t/25175/7,2A423C4C-B3E0-4C9C-95D1-A0A86ACE1465,"Overheated, Failed Brakes",2005-04-20,3156,NaN,"[yes, guy, plenty, time, go, around, land, wit...",NaN,Forum,2,0.000079,2300
3,brakes during taxi,https://forum.cirruspilots.org/t/68466/1,AB750CAD-7103-48DC-BB4A-8B82B792B33E,Brake fire,2013-06-14,14208,NaN,"[got, fresh, annual, taxi, brake, clamped, nec...",NaN,Forum,2,0.000173,14
4,brakes during taxi,https://forum.cirruspilots.org/t/26578/1,97F47B9B-F055-4F1D-9C56-8B5FF2A3D32C,Brake Failure and Fire,2005-08-12,4281,NaN,"[today, right, landing, gear, aircraft, catch,...",NaN,Forum,2,0.000081,340
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,music volume low,https://forum.cirruspilots.org/t/8776/4,72DC0271-E2CE-4BFA-90CE-2E3DFD58C186,Garmin 340 Volume of Auxiliary Entertainment I...,2008-07-29,2130,NaN,"[patrick, way, hear, atc, transmission, kid, t...",NaN,Forum,2,0.000101,55119
1029,music volume low,https://forum.cirruspilots.org/t/28479/2,B220B5CB-B93E-483F-B9DF-32F00B6FB5F7,Clarity Aloft + Bose = Very Quiet,2006-01-10,4127,NaN,"[always, good, idea, wear, plug, underneath, r...",NaN,Forum,2,0.000083,2748
1030,music volume low,https://forum.cirruspilots.org/t/46807/4,67F69D32-5AE8-4E3B-9066-B7EB36FB7138,Faint transmit on the Perspective and faint XM...,2009-03-01,2166,NaN,"[quote, userbob, anderson, quote, usercarl, la...",NaN,Forum,2,0.000107,4446
1031,music volume low,https://forum.cirruspilots.org/t/22290/9,2AE957AA-EC19-458F-9262-DE37DA78C77E,Audio - again,2004-09-08,2107,NaN,"[reply, toi, read, previous, thread, audio, in...",NaN,Forum,2,0.000076,25042


time: 30.9 ms


In [20]:
model.init_sims(replace=True) # normalize the word2vec vector
all_query_df['WMDistance'] = all_query_df.apply(lambda x: model.wmdistance(x['Query'], x['FormattedBody']), axis=1)

time: 25.9 s


In [21]:
all_query_df['ContentScore'] = all_query_df['WMDistance'].rank(ascending=False)

time: 10.4 ms


In [22]:
all_query_df['FormattedBody'] = all_query_df['FormattedBody'].apply(lambda x: ' '.join(x))

time: 8.97 ms


In [23]:
all_query_df.head()

,Query,url,ContentId,fancy_title,Date,UserID,Title,FormattedBody,TotalViews,Source,SourceScore,RecencyRate,AuthorScore,WMDistance,ContentScore
0,brakes during taxi,https://forum.cirruspilots.org/t/41040/5,6D601E0C-14D7-41DB-BAEF-3A6333537AB8,Cirrus SR22 Brake problems,2012-11-21,3695,NaN,quote userdavid martin robert thats mechanic t...,NaN,Forum,2,0.000160,19536,1.291631,803.0
1,brakes during taxi,https://forum.cirruspilots.org/t/65942/2,ADCD2721-0351-4DD7-8B07-AE423D3420F0,cirrus sr22 brake wear,2012-12-11,10652,NaN,8,NaN,Forum,2,0.000161,19902,1.323552,289.0
2,brakes during taxi,https://forum.cirruspilots.org/t/25175/7,2A423C4C-B3E0-4C9C-95D1-A0A86ACE1465,"Overheated, Failed Brakes",2005-04-20,3156,NaN,yes guy plenty time go around land without inc...,NaN,Forum,2,0.000079,2300,1.307057,586.0
3,brakes during taxi,https://forum.cirruspilots.org/t/68466/1,AB750CAD-7103-48DC-BB4A-8B82B792B33E,Brake fire,2013-06-14,14208,NaN,got fresh annual taxi brake clamped necessitat...,NaN,Forum,2,0.000173,14,1.327266,217.0
4,brakes during taxi,https://forum.cirruspilots.org/t/26578/1,97F47B9B-F055-4F1D-9C56-8B5FF2A3D32C,Brake Failure and Fire,2005-08-12,4281,NaN,today right landing gear aircraft catch fire s...,NaN,Forum,2,0.000081,340,1.333966,122.0


time: 15.9 ms


__Re-rank search results__

In [24]:
def topic_match(search_query):
    '''
    Extract the topic of searching query by LDA model trained by forum posts
    '''
    texts1 = [[word for word in doc.lower().split() if word not in stop_words] for doc in [search_query]]
    corpus1 = [id2word.doc2bow(t) for t in texts1]
    result = lda_model.get_document_topics(corpus1)
    LDA_topic = max(result[0], key=lambda x: x[1])
    
    return LDA_topic

time: 2 ms


In [29]:
def keyword_from_query(query):
    '''
    Extract keywords from query
    '''
    query_word = [word for word in query.split() if word not in stop_words]
    
    return query_word

time: 1.02 ms


In [26]:
def query_hit_ratio(query_word, post):
    '''
    Return percent of the query that are contained in the posts
    '''
    len_query = len(query_word)
    count_hit = 0
    
    for word in query_word:
        if word in post:
            count_hit += 1
    
    ratio = count_hit / len_query

    return ratio

time: 919 µs


In [30]:
def hit_WordPairs(query_word, wordPairs):
    '''
    Return a list of pre-defined word pairs that are contained by the search query
    '''
    query_string = ' '.join(query_word)
    matched_wordpair = list()
    for wordpair in wordPairs:
        if query_string.lower().count(wordpair) > 0:
            matched_wordpair.append(wordpair)
    return matched_wordpair

time: 999 µs


In [31]:
def ratio_wordpair_match(post):
    '''
    Count the ratio of word pair matching between search query and posts
    '''
    summa = 0
    
    matched_wordpairs = hit_WordPairs(query_word, wordPairs)
    
    if len(matched_wordpairs) != 0:
        for matched_wordpair in matched_wordpairs:
            ratio = post.lower().count(matched_wordpair) / len(post)
            summa += ratio
            
    return summa

time: 920 µs


In [32]:
def ratio_word_match(post):
    '''
    Count the ratio of word matching between search query and posts
    '''
    summa = 0
    
    for word in list(set(query_word)):
        ratio = post.lower().count(word) / len(post)
        summa += ratio
    
    return summa

time: 994 µs


In [33]:
def count_word_match(post):
    '''
    Count the word matching between search query and posts
    '''
    summa = 0
    
    for word in list(set(query_word)):
        count = post.lower().count(word)
        summa += count
    
    return summa

time: 990 µs


In [36]:
def Normalization(col, df):
    '''
    Normalize data in three attributes
    '''
    nomolized_col = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    
    return nomolized_col

time: 997 µs


In [40]:
def assign_weights(weights, df):
    '''
    Assign different sets of weights to three attributes
    '''
    df_copy = df.copy()
    df_copy['TotalScore'] = df['RecencyRate'] * weights[0] + df['AuthorScore'] * weights[1] + df['TotalContentScore'] * weights[2]
    df_copy.sort_values("TotalScore", inplace = True, ascending=False)
    
    return df_copy

time: 1.03 ms


In [41]:
def reset_index(df):
    '''
    Reset the index of a dataframe
    '''
    df.reset_index(drop=True, inplace=True)
    
    return df

time: 998 µs


In [42]:
comb1 = ''
comb2 = ''
comb3 = ''

query_list = list(all_query_df['Query'].unique())
for query in query_list:
    # Take the subset of search results from the query
    sub_dataset = all_query_df[all_query_df['Query'] == query]
    
    query_word = keyword_from_query(query)
    wordPairs = list(map(lambda x: x.lower(),list(pd.read_csv('WordPairs.csv')['Word Pairs'])))
    sub_dataset['word_Match'] = sub_dataset['FormattedBody'].apply(ratio_word_match)
    sub_dataset['wordpair_Match'] = sub_dataset['FormattedBody'].apply(ratio_wordpair_match)
    sub_dataset['querywords_hit_ratio'] = sub_dataset.apply(lambda x: query_hit_ratio(query_word, x['FormattedBody']), axis=1)
    
    sub_dataset['Match'] = sub_dataset['word_Match'] + sub_dataset['wordpair_Match'] + sub_dataset['querywords_hit_ratio']
    
    # Normalize data
    for col in ['RecencyRate', 'AuthorScore', 'ContentScore', 'Match']:
        sub_dataset.loc[:,col] = Normalization(col, sub_dataset)
    
    sub_dataset['TotalContentScore'] = 0.5 * sub_dataset['ContentScore'] + 0.5 * sub_dataset['Match']
    
    # Apply three sets of weights that add up to the total score
    temp1 = assign_weights([1/4, 2/9, (1-1/4-2/9)], sub_dataset)
    temp2 = assign_weights([1/5, 3/10, 1/2], sub_dataset)
    temp3 = assign_weights([3/10, 1/10, 6/10], sub_dataset)
    #temp3 = assign_weights([0, 0, 1], sub_dataset)
    
    if type(comb1) == str:
        comb1 = temp1
        comb2 = temp2
        comb3 = temp3
    else:
        # Combine search results for all search queries
        comb1 = pd.concat([comb1, temp1])
        comb2 = pd.concat([comb2, temp2])
        comb3 = pd.concat([comb3, temp3])

for df in [comb1, comb2, comb3]:
    reset_index(df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

time: 312 ms


In [43]:
comb1

,Query,url,ContentId,fancy_title,Date,UserID,Title,FormattedBody,TotalViews,Source,...,RecencyRate,AuthorScore,WMDistance,ContentScore,word_Match,wordpair_Match,querywords_hit_ratio,Match,TotalContentScore,TotalScore
0,brakes during taxi,https://forum.cirruspilots.org/t/94283/1,5BC85C26-D659-459A-854B-99E14B45A2FF,Beringer brake pad wear,2018-12-20,16970,NaN,dual caliper beringer brake sr22 g3 don‘t „rid...,NaN,Forum,...,0.407844,0.048787,1.284850,0.835644,0.004785,0.0,1.0,0.999691,0.917667,0.597127
1,brakes during taxi,https://forum.cirruspilots.org/t/98008/90,B6E67966-7CE4-465F-80F8-2E817735A523,"CAPS Save #91 in SR22 N578SP near Fort Wayne, ...",2019-10-01,2256,NaN,reading earlier cap discussion rich kalgaard e...,NaN,Forum,...,1.000000,0.530518,1.294186,0.755446,0.000000,0.0,0.0,0.000000,0.377723,0.567247
2,brakes during taxi,https://forum.cirruspilots.org/t/62871/1,20C67829-C5EB-45BE-B802-6E31D5A5A81F,Vibration while taxiing - suggestions for what...,2012-04-23,13923,NaN,realize difficult diagnose problem forum postb...,NaN,Forum,...,0.044508,0.005528,1.288497,0.804950,0.005096,0.0,1.0,1.000000,0.902475,0.488662
3,brakes during taxi,https://forum.cirruspilots.org/t/19650/3,AD2ED868-83CB-4404-86EF-0072EB200FD6,frozen brakes,2004-01-09,2130,NaN,somthing add say able pullout hanger without p...,NaN,Forum,...,0.006876,0.982811,1.313584,0.450495,0.004049,0.0,0.5,0.501493,0.475994,0.471340
4,brakes during taxi,https://forum.cirruspilots.org/t/92794/2,2A2DD3CE-B92E-420C-99A4-73EB272CB5EB,SR22 [Brakes],2018-08-27,21869,NaN,quote useralexander wolf alexander wolf wrote ...,NaN,Forum,...,0.325458,0.011483,1.267987,0.940594,0.004405,0.0,0.5,0.501848,0.721221,0.464561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,music volume low,https://forum.cirruspilots.org/t/44997/20,B07CE56F-C8AF-4CAF-BDD4-215BB52C1A30,Impedance matching device (adapter),2008-12-08,3406,NaN,mario one old plane worked greatemail frankfot...,NaN,Forum,...,0.048722,0.001748,1.319507,0.313387,0.000000,0.0,0.0,0.000000,0.156694,0.095268
1029,music volume low,https://forum.cirruspilots.org/t/22350/13,8DA1DC63-ADAA-44B4-8A89-570E0C6C233D,Roady2,2004-09-17,2363,NaN,new roady2 work well plane know original roady...,NaN,Forum,...,0.015341,0.086355,1.334550,0.065923,0.000000,0.0,0.0,0.000000,0.032961,0.040422
1030,music volume low,https://forum.cirruspilots.org/t/16506/3,9DD8C55D-4E65-474A-B4C2-B3239C950C71,cd player,2003-06-12,3253,NaN,searched searched site impedance schematic wit...,NaN,Forum,...,0.008584,0.002925,1.330927,0.113083,0.000000,0.0,0.0,0.000000,0.056542,0.032637
1031,music volume low,https://forum.cirruspilots.org/t/16506/3,9DD8C55D-4E65-474A-B4C2-B3239C950C71,cd player,2003-06-12,3253,NaN,searched searched site impedance schematic wit...,NaN,Forum,...,0.008584,0.002925,1.330927,0.113083,0.000000,0.0,0.0,0.000000,0.056542,0.032637


time: 24.8 ms


In [44]:
comb1[['Query', 'url', 'ContentId', 'fancy_title']].to_csv('wmd_ratio_comb1.csv') # 1/4, 2/9, (1-1/4-2/9)
comb2[['Query', 'url', 'ContentId', 'fancy_title']].to_csv('wmd_ratio_comb2.csv') # 1/5, 3/10, 1/2
comb3[['Query', 'url', 'ContentId', 'fancy_title']].to_csv('wmd_ratio_comb3.csv') # 3/10, 1/10, 6/10

time: 38.9 ms
